Primero importaremos las librerías que utilizaremos para procesar la imágenes

In [ ]:
# Para visualización:
import matplotlib.pyplot as plt
# Para manipulación de los pixeles a través de K-means
from sklearn.cluster import KMeans
# Para trabajar con matrices
import numpy as np

# Funciones:

Desarrollaremos 4 funciones para el proceso de obtener la superficie aproximada:

1. Funcion Quitar, esta función la usaremos para eliminar zonas de ruido en las imágenes.

2. Función segmentar_imagen_RGB, esta será para generar una matriz con los pixeles segmentados por el método K-means en escala RGB

3. Función aproximar_Areas, realiza un proceso similar a la función anterior pero se utilizará para entregar el area en km².

4. Función visualizar_aproximaciones, la utilizaremos para visualizar gráficamente la segmentación de las imágenes.

In [ ]:
def quitar(imagen):
    #Elaboramos función para eliminar una sombra que nos genera un mayor ruido en la imagen

    x_inicio = 200
    x_fin = 403
    y_inicio = 273
    y_fin = 573

    # Creamos una imagen en blanco del mismo tamaño que la original
    imagen_blancos = np.copy(imagen)

    # Asignamos el color blanco a los píxeles en el rango especificado
    imagen_blancos[y_inicio:y_fin, x_inicio:x_fin] = [255, 255, 255]
    imagen_blancos[139:273, 249:403]= [255, 255, 255]
    imagen_blancos[0:251, 0:71]= [255, 255, 255]
    imagen_blancos[100:571, 300:403 ]= [255, 255, 255]
    imagen_blancos[300:571,183:403]= [255, 255, 255]
    imagen_blancos[547:571, 0:403]= [255, 255, 255]
    imagen_blancos[0:23, 0:403]= [255, 255, 255]
    imagen_blancos[0:573, 0:20]= [255, 255, 255]
    imagen_blancos[0:104, 0:150]= [255, 255, 255]
    imagen_blancos[220:571, 205:403]= [255, 255, 255]
    #Retornamos la imagen nueva
    return imagen_blancos

In [ ]:
def segmentar_imagen_RGB(ruta_imagen, num_clusters, colores_cluster):
    # Definimos una función para realizar la matriz de pixeles por K-medias en escala RGB

    # Cargamos la imagen
    imagen = plt.imread(ruta_imagen)

    # Obtenenos las dimensiones de la imagen
    alto, ancho, _ = imagen.shape

    # Convertimos la imagen en una matriz de píxeles
    # Primero debemos recortar la imagen
    imagen2 = quitar(imagen)
    matriz_pixels = imagen2.reshape(-1, 3)

    # Realizamos la segmentación por pixeles por K-medias
    kmeans = KMeans(n_clusters=num_clusters, n_init=10)
    kmeans.fit(matriz_pixels)

    # Tamaño del cluster con mayor cantidad de píxeles
    frecuencias = np.bincount(kmeans.labels_)
    cluster_mas_grande = np.argmax(frecuencias)
    frecuencias[cluster_mas_grande] = 0
    segundo_cluster_mas_grande = np.argmax(frecuencias)
    mascara = (kmeans.labels_ == segundo_cluster_mas_grande)
    mascara = mascara.reshape(alto, ancho)


    # Asignamos colores RGB a la imagen ya segmentada
    imagen_nueva = colores_cluster[kmeans.labels_].reshape(alto, ancho, 3)

    # Creamos una nueva imagen dejando todo lo diferente al Lago en Blanco
    imagen_nueva2 = imagen_nueva.copy()
    imagen_nueva2[~mascara] = [255, 255, 255]  # Establecemos los píxeles no deseados en blanco
    return imagen_nueva2

In [ ]:
def aproximar_areas(ruta_imagen, num_clusters):
    # Creamos función similar a la anterior, sin embargo en esta se retornara el area aproximada del lago
    # Cargamos la imagen
    imagen = plt.imread(ruta_imagen)

    # Obtenemos las dimensiones de la imagen
    alto, ancho, _ = imagen.shape

    # Convertimos la imagen en una matriz de píxeles
    # Primero debemos recortar la imagen
    imagen2 = quitar(imagen)
    matriz_pixels = imagen2.reshape(-1, 3)


    # Realizamos la segmentación por K-medias
    kmeans = KMeans(n_clusters=num_clusters, n_init=10)
    kmeans.fit(matriz_pixels)

    # Calculamos el tamaño del cluster más grande
    frecuencias = np.bincount(kmeans.labels_)
    cluster_mas_grande = np.argmax(frecuencias)
    frecuencias[cluster_mas_grande] = 0
    segundo_cluster_mas_grande = np.argmax(frecuencias)
    tamaño_lago_Caburgua_en_pixel = np.sum(kmeans.labels_ == segundo_cluster_mas_grande)

    # Calculamos las medidas pixel en km
    ancho_imagen= 17.33 #ancho en km de la imagen
    alto_imagen= 12.79 #alto en km de la imagen
    area_imagen = ancho_imagen* alto_imagen # Corresponde al área de cada pixel en km²
    total_pixeles = ancho*alto
    tamaño_lago_Caburgua = area_imagen*tamaño_lago_Caburgua_en_pixel/total_pixeles

    # Imprimimos  la superficie del lago en metros cuadrados
    print(f"[{i+1}] Superficie del lago en km²:", tamaño_lago_Caburgua, "(", tamaño_lago_Caburgua_en_pixel,"pixeles)")

    # Retornamos el tamaño del lago
    return tamaño_lago_Caburgua


In [ ]:
def visualizar_aproximaciones(imagenes):
    # Creamos una funcion para visualizar todas las aproximaciones
    plt.figure(figsize=(12, 6))  # Establecemos el tamaño del gráfico

    for k, imagen in enumerate(imagenes):
        plt.subplot(3, 5, k + 1)  # Creamos una cuadrícula de subgráficos de 3 filas y 5 columnas
        plt.imshow(imagen)
        plt.title(f'Iteración {k + 1}')
        plt.axis('off')

    plt.tight_layout()  # Ajustamos el diseño de los subgráficos
    plt.show() # Mostramos el gráfico completo

Importamos las imágenes y asigmanos la cantidad de clusters:

In [ ]:
rutas = [
    "/content/1984.jpeg", "/content/1985.jpeg", "/content/1986.jpeg",
    "/content/1987.jpeg", "/content/1988.jpeg", "/content/1989.jpeg",
    "/content/1990.jpeg", "/content/1991.jpeg", "/content/1992.jpeg",
    "/content/1993.jpeg", "/content/1994.jpeg", "/content/1995.jpeg",
    "/content/1996.jpeg", "/content/1997.jpeg", "/content/1998.jpeg",
    "/content/1999.jpeg", "/content/2000.jpeg", "/content/2001.jpeg",
    "/content/2002.jpeg", "/content/2003.jpeg", "/content/2004.jpeg",
    "/content/2005.jpeg", "/content/2006.jpeg", "/content/2007.jpeg",
    "/content/2008.jpeg", "/content/2009.jpeg", "/content/2010.jpeg",
    "/content/2011.jpeg", "/content/2012.jpeg", "/content/2013.jpeg",
    "/content/2014.jpeg", "/content/2015.jpeg", "/content/2016.jpeg",
    "/content/2017.jpeg", "/content/2018.jpeg", "/content/2019.jpeg",
    "/content/2020.jpeg", "/content/2021.jpeg", "/content/2022.jpeg"
]


num_clusters = 5

Este código hara 15 iteraciones por imágen y seleccionara la mediana de las superficies aproximadas para asignar el valor aproximado para cada año.

In [ ]:
# Realizamos múltiples iteraciones y almacenamos los resultados
np.random.seed(47)
colores_centroides=np.random.randint(0, 256, size=(num_clusters, 3))
iteraciones_pedidas = 15
medidas = [] # Lista vacia para almacenar las aproximaciones de las áreas
imagenes = [] # Lista vacía para almacenar los gráficos aproximados
aproximaciones = []

for j in range(len(rutas)):
    ruta_imagen= rutas[j]
    for i in range(iteraciones_pedidas):
        #Aplicamos las funciones
        imagen_segmentada = segmentar_imagen_RGB(ruta_imagen, num_clusters, colores_centroides)
        imagenes.append(imagen_segmentada)
        medidas.append(aproximar_areas(ruta_imagen, num_clusters))

    visualizar_aproximaciones(imagenes)
    imagenes=[]

    # Calculamos la mediana de las medidas obtenidas
    Area = np.median(medidas)
    medidas=[]
    aproximaciones.append(Area)

    # Imprimimos el resultado final
    print(f"Área aproximada del lago en kilómetros cuadrados:", Area)

Mostrar todas las aproximaciones:

In [ ]:
print(aproximaciones)